In [ ]:
!pip install --upgrade gensim

In [ ]:
import gensim.downloader
import pickle
import numpy as np
from sklearn import metrics
import pandas as pd
import tensorflow as tf

In [ ]:
embeddings = gensim.downloader.load('word2vec-google-news-300')

[==============================--------------------] 60.1% 999.5/1662.8MB downloaded

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
with open("./drive/MyDrive/school/embeddings.pkl", "rb") as f:
  embeddings = pickle.load(f)

UnpicklingError: ignored

Q1.1 Based on word2vec embeddings you have downloaded, use cosine similarity to find the most similar
word to each of these words: (a) “student”; (b) “Apple”; (c) “apple”. Report the most similar word
and its cosine similarity.

In [ ]:
print(embeddings.most_similar('student')[0])
print(embeddings.most_similar('Apple')[0])
print(embeddings.most_similar('apple')[0])

('students', 0.7294867038726807)
('Apple_AAPL', 0.7456986308097839)
('apples', 0.720359742641449)


In [ ]:
student_embeddings = embeddings['student'].reshape(1,300)
students_embeddings = embeddings['students'].reshape(1,300)
print(metrics.pairwise.cosine_similarity(student_embeddings, students_embeddings))

[[0.72948664]]


(a) Describe the size (number of sentences) of the training, development and test file for CoNLL2003.
Specify the complete set of all possible word labels based on the tagging scheme (IO, BIO,
etc.) you chose.

processing data

In [ ]:
header_names = ['word', 'useless1', 'useless2', 'tag']
train_raw = pd.read_csv("./drive/MyDrive/school/eng.train", header=None, sep=' ', names=header_names, skip_blank_lines=False)
validation_raw = pd.read_csv("./drive/MyDrive/school/eng.testa", header=None, sep=' ', names=header_names, skip_blank_lines=False)
test_raw = pd.read_csv("./drive/MyDrive/school/eng.testb", header=None, sep=' ', names=header_names, skip_blank_lines=False)

In [ ]:
train_raw[:20]

In [ ]:
null_indices = train_raw[train_raw['word'].isnull()].reset_index()['index']
null_indices

0             9
1            12
2            15
3            46
4            78
          ...  
14984    219535
14985    219538
14986    219543
14987    219546
14988    219551
Name: index, Length: 14989, dtype: int64

In [ ]:
test_raw

,word,useless1,useless2,tag
0,SOCCER,NN,I-NP,O
1,-,:,O,O
2,JAPAN,NNP,I-NP,I-LOC
3,GET,VB,I-VP,O
4,LUCKY,NNP,I-NP,O
...,...,...,...,...
50344,",",",",O,O
50345,Bobby,NNP,I-NP,I-PER
50346,.,.,O,O
50347,NaN,NaN,NaN,NaN


In [ ]:
validation_raw

,word,useless1,useless2,tag
0,CRICKET,NNP,I-NP,O
1,-,:,O,O
2,LEICESTERSHIRE,NNP,I-NP,I-ORG
3,TAKE,NNP,I-NP,O
4,OVER,IN,I-PP,O
...,...,...,...,...
55038,Dhaka,NNP,I-NP,I-ORG
55039,Newsroom,NNP,I-NP,I-ORG
55040,880-2-506363,CD,I-NP,O
55041,NaN,NaN,NaN,NaN


In [ ]:
def drop_na_labels(data):
  temp = data[['word', 'tag']]
  #without_na = temp.dropna(subset=['tag']).reset_index().drop(columns='index')
  return temp

In [ ]:
def get_tag_to_integer_dict(train_without_na):
  count = 0
  tag_to_integer_dictionary = {}
  for tag in train_without_na['tag'].unique():
    tag_to_integer_dictionary[tag] = count
    count += 1

  return tag_to_integer_dictionary

In [ ]:
def get_tag_to_integer_dict_from_raw(train_raw):
  temp = train_raw[['word', 'tag']]
  without_na = temp.dropna(subset=['tag']).reset_index().drop(columns='index')
  tag_to_integer_dictionary = get_tag_to_integer_dict(without_na)
  return tag_to_integer_dictionary

In [ ]:
tag_to_integer_dictionary = get_tag_to_integer_dict_from_raw(train_raw)

In [ ]:
def get_sentences_wordstags_array(train_without_na):
  sentences_train = []
  new_sentence = []

  for i, row in train_without_na.iterrows():

    if row.isna().all():
      sentences_train.append(new_sentence)
      if len(new_sentence) > 80:
        print("long sentence: ", new_sentence)
      new_sentence = []

    else:
      new_sentence.append([row[0], row[1]])


    if i > 999999:
      print("more than 999999")
      break

  return sentences_train

In [ ]:
def get_sentences_embeddingstags_array(sentences_wordstags_array, tag_to_integer_dictionary):
  # replace words with embeddings and tags with integers
  embeddings_in_sentences = []
  new_sentence = []

  count = 0
  for sentence in sentences_wordstags_array:
    for word, tag in sentence:
      if tag not in tag_to_integer_dictionary:
        #print(f"{tag} not found with {word}, skipping")
        continue
      if word not in embeddings:
        new_embedding = np.random.rand(1, 300)[0].astype(np.float32)

      else:
        new_embedding = embeddings[word].astype(np.float32)

      new_sentence.append([new_embedding, tag_to_integer_dictionary[tag]])
      count += 1

    embeddings_in_sentences.append(new_sentence)
    new_sentence = []

    if count > 999999:
      print("more than 999999")
      break

  return embeddings_in_sentences

In [ ]:
def get_without_labels(sentences_embeddingstags):
  # create copy that doesnt have the NER tag
  without_label = []
  new_sentence = []

  count = 0
  for sentence in sentences_embeddingstags:
    for embedding, tag in sentence:
      new_sentence.append(embedding)
      count += 1

    without_label.append(new_sentence)
    new_sentence = []

    if count > 999999:
      print("more than 999999")
      break

  return without_label

In [ ]:
def get_labels(sentences_embeddingstags):
  # create labels
  labels = []
  new_sentence = []

  count = 0
  for sentence in sentences_embeddingstags:
    for embedding, tag in sentence:
      new_sentence.append(tag)
      count += 1

    labels.append(np.array(new_sentence))
    new_sentence = []

    if count > 999999:
      print("more than 999999")
      break

  return labels

In [ ]:
def pad_without_labels_and_labels(without_labels, labels, max_sentence_length):
  # padding
  print("padding: ", max_sentence_length)
  padded = tf.keras.utils.pad_sequences(without_labels, padding="post", dtype="float32", maxlen=max_sentence_length)
  padded_labels = tf.keras.utils.pad_sequences(labels, padding="post", maxlen=max_sentence_length)

  return padded, padded_labels

In [ ]:
def convert_padded_and_padded_labels_to_np_arrays(padded, padded_labels):
  # input: [batch, timestep, feature]
  padded_np = np.array(padded)
  padded_labels_np = np.array(padded_labels)
  """
  print(type(train_labels))
  print(type(train_labels[0]))
  print(type(train_labels[0][0]))
  print(type(train))
  print(type(train[0]))
  print(type(train[0][0]))
  print(type(train[0][0][0]))
  """
  return padded_np, padded_labels_np



In [ ]:
def process_raw_to_input_and_labels(raw, tag_to_integer_dictionary):
  without_na = drop_na_labels(raw)
  num_classes = len(without_na['tag'].unique())
  sentences_wordstags = get_sentences_wordstags_array(without_na)
  sentences_embeddingstags = get_sentences_embeddingstags_array(sentences_wordstags, tag_to_integer_dictionary)
  without_labels = get_without_labels(sentences_embeddingstags)
  labels = get_labels(sentences_embeddingstags)

  return without_labels, labels, tag_to_integer_dictionary, num_classes

In [ ]:
def pad_input_and_labels(without_labels, labels, max_sentence_length):
  padded, padded_labels = pad_without_labels_and_labels(without_labels, labels, max_sentence_length)
  padded_np, padded_labels_np = convert_padded_and_padded_labels_to_np_arrays(padded, padded_labels)

  return padded_np, padded_labels_np

In [ ]:
unpadded_train, unpadded_train_labels, tag_to_integer_dictionary, num_classes = process_raw_to_input_and_labels(train_raw, tag_to_integer_dictionary)
unpadded_val, unpadded_val_labels, unused_val_dictionary, unused_val_num_classes = process_raw_to_input_and_labels(train_raw, tag_to_integer_dictionary)
unpadded_test, unpadded_test_labels, unused_test_dictionary, unused_test_num_classes = process_raw_to_input_and_labels(test_raw, tag_to_integer_dictionary)

max_sentence_length = 0
for sentence in unpadded_train:
  if len(sentence) > max_sentence_length:
    max_sentence_length = len(sentence)
for sentence in unpadded_val:
  if len(sentence) > max_sentence_length:
    max_sentence_length = len(sentence)
for sentence in unpadded_test:
  if len(sentence) > max_sentence_length:
    max_sentence_length = len(sentence)

train, train_labels = pad_input_and_labels(unpadded_train, unpadded_train_labels, max_sentence_length)
val, val_labels = pad_input_and_labels(unpadded_val, unpadded_val_labels, max_sentence_length)
test, test_labels = pad_input_and_labels(unpadded_test, unpadded_test_labels, max_sentence_length)

In [ ]:
import gc
gc.collect()

33383

In [ ]:
import sys

# Function to convert bytes to a more readable format
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

# Iterate over the dictionary of local variables
for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key=lambda x: -x[1])[:10]:  # adjust slice for more or fewer variables
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))


                         train:  2.1 GiB
                           val:  2.1 GiB
                          test: 522.6 MiB
                            __: 12.2 MiB
                validation_raw: 12.2 MiB
                           _12: 12.2 MiB
                           ___: 11.1 MiB
                      test_raw: 11.1 MiB
                           _11: 11.1 MiB
                  train_labels:  7.1 MiB


In [ ]:
print(train.shape)
print(test.shape)

(14986, 124, 300)
(3683, 124, 300)


In [ ]:
print(num_classes)
num_classes = 8 #fix this

9


In [ ]:
import keras
from keras import layers

In [ ]:
inputs = keras.Input(shape = (train.shape[1], train.shape[2]))
x = layers.Masking(mask_value=0.0)(inputs)
x = layers.Bidirectional(layers.LSTM(16, return_sequences=True))(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 124, 300)]        0         
                                                                 
 masking (Masking)           (None, 124, 300)          0         
                                                                 
 bidirectional (Bidirection  (None, 124, 32)           40576     
 al)                                                             
                                                                 
 dense (Dense)               (None, 124, 8)            264       
                                                                 
Total params: 40840 (159.53 KB)
Trainable params: 40840 (159.53 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
print(type(train[0][0][0]))
[print(i.shape, i.dtype) for i in model.inputs]
[print(o.shape, o.dtype) for o in model.outputs]
[print(l.name, l.input_shape, l.dtype) for l in model.layers]

<class 'numpy.float32'>
(None, 124, 300) <dtype: 'float32'>
(None, 124, 8) <dtype: 'float32'>
input_1 [(None, 124, 300)] float32
masking (None, 124, 300) float32
bidirectional (None, 124, 300) float32
dense (None, 124, 32) float32


[None, None, None, None]

In [ ]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
#model.fit(train, train_labels, batch_size=32, epochs=2)
model.fit(train, train_labels, batch_size=32, epochs=2, validation_data=(val, val_labels))